In [1]:
import json
from arango import ArangoClient
from langchain_community.graphs import ArangoGraph

# change tenant_id
tenant_id = 'ke'
NODE_COLLECTION = 'nodes'
RELATIONSHIP_COLLECTION = 'relationships'

client = ArangoClient(hosts='http://localhost:8529')

sys_db = client.db(
    '_system',
    username = 'root',
    password = ''
   )

db = client.db(tenant_id)
graph_db = ArangoGraph(db)
# print(json.dumps(graph_db.schema, indent=4, ensure_ascii=False))

In [7]:
entity_id = "fpt smart cloud"
entity = graph_db.query(f"""
                FOR n IN {NODE_COLLECTION}
                FILTER n._query_name == '{entity_id.lower()}'
                RETURN n._id
               """)
# print(json.dumps(result, indent=4, ensure_ascii=False))
print(entity)

['nodes/0fd5d0ae-f2c3-56d7-85ab-fb0469062afd', 'nodes/75acaa43-d674-5571-91d7-604528346979']


In [8]:

result = graph_db.query(f"""
                FOR r IN {RELATIONSHIP_COLLECTION}
                FILTER r._from in {entity}
                OR r._to == 'nodes/c43854b8-f01f-5667-aac5-8bd7ca88b1da'
                RETURN r.triplet_sentence
               """)
print(result)

['FPT AI có giải pháp FPT AI Chat, nền tảng hội thoại thông minh cho phép tạo lập và quản lý chatbot tự động giao tiếp hai chiều với khách hàng.', 'FPT AI cung cấp giải pháp FPT AI Engage, nền tảng trợ lý ảo tổng đài thực hiện cuộc gọi tự động hai chiều với giọng nói tự nhiên.', 'FPT AI sở hữu giải pháp FPT AI Read, nền tảng nhận diện và trích xuất thông tin từ ảnh chụp với khả năng xử lý đa dạng mọi loại văn bản.', 'FPT AI có giải pháp FPT AI Enhance, cho phép tổng đài tự động toàn diện quy trình quản lý chất lượng và theo dõi các chỉ số chuyên sâu theo thời gian thực.', 'FPT AI cung cấp giải pháp FPT AI eKYC, khả năng định danh khách hàng hoàn chỉnh với nhiều cấp độ bảo mật.', 'FPT AI giúp nâng cao hiệu suất xử lý công việc, mang lại sự kết hợp hoàn hảo giữa người và máy.', 'FPT AI gắn kết khách hàng hiệu quả thông qua việc tiếp cận, tương tác và thu hút tối đa khách hàng.', 'FPT AI kiến tạo giá trị mới cho khách hàng thông qua việc thiết lập những trải nghiệm mới và thói quen sử dụn

In [14]:
for i in result:
    print(i)

FPT AI có giải pháp FPT AI Chat, nền tảng hội thoại thông minh cho phép tạo lập và quản lý chatbot tự động giao tiếp hai chiều với khách hàng.
FPT AI cung cấp giải pháp FPT AI Engage, nền tảng trợ lý ảo tổng đài thực hiện cuộc gọi tự động hai chiều với giọng nói tự nhiên.
FPT AI sở hữu giải pháp FPT AI Read, nền tảng nhận diện và trích xuất thông tin từ ảnh chụp với khả năng xử lý đa dạng mọi loại văn bản.
FPT AI có giải pháp FPT AI Enhance, cho phép tổng đài tự động toàn diện quy trình quản lý chất lượng và theo dõi các chỉ số chuyên sâu theo thời gian thực.
FPT AI cung cấp giải pháp FPT AI eKYC, khả năng định danh khách hàng hoàn chỉnh với nhiều cấp độ bảo mật.
FPT AI giúp nâng cao hiệu suất xử lý công việc, mang lại sự kết hợp hoàn hảo giữa người và máy.
FPT AI gắn kết khách hàng hiệu quả thông qua việc tiếp cận, tương tác và thu hút tối đa khách hàng.
FPT AI kiến tạo giá trị mới cho khách hàng thông qua việc thiết lập những trải nghiệm mới và thói quen sử dụng mới giữa doanh nghiệp

In [5]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_template = """# GOAL
Extract all entities from the following TEXT INPUT. 
Entities belong to one of the following topics: Person, Organization, Event, Product and Services, Location, Time,...

# EXAMPLE
"Bà Đỗ Mỹ Linh quê ở đâu?"
Output: ["Đỗ Mỹ Linh"]

EXAMPLE
"Ông Kim Ri Cha là giám đốc công ty cổ phần RICONS"
Output: ["Kim Ri Cha", "RICONS"]

Extract all entities based on TEXT INPUT below and following EXAMPLE above:
"{input}"

# Output: 
Only return as list of entities in the format below:
["entity_1", "entity_2", ..., "entity_n"]"""

prompt = PromptTemplate.from_template(prompt_template)

llm_extracttion = ChatOpenAI(model='gpt-4o-mini', temperature=0.1, api_key=openai_api_key)
entity_extraction_chain = (
    prompt
    | llm_extracttion
    | StrOutputParser()
)

prompt_template = """# TASK
Answers QUESTION based on the CONTEXT provided below:
-- start_context --

{context}

-- end_context --

# INSTRUCTIONS
- Please present your answer clearly based on the given CONTEXT.
- Do not fabricate answer content, only answer based on CONTEXT.

Answer the following QUESTION based on the given CONTEXT and the above INSTRUCTIONS:
"{question}"

Answer:"""
prompt = PromptTemplate.from_template(prompt_template)

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)
answer_chain = (
    prompt
    | llm
    | StrOutputParser()
)

def get_triplets_by_entites(entites):
    triplets = list()
    for entity in entites:
        node_ids = graph_db.query(f"""
                FOR n IN {NODE_COLLECTION}
                FILTER n._query_name == '{entity.lower()}'
                RETURN n._id
               """)
        if len(node_ids) == 0: 
            continue
        for node_id in node_ids:
            result = graph_db.query(f"""
                    FOR r IN {RELATIONSHIP_COLLECTION}
                    FILTER r._from == '{node_id}' 
                    OR r._to == '{node_id}'
                    RETURN r.triplet_sentence
                """)
            triplets.extend(result)
    return triplets

def get_answer(query):
    input = {'input': query}
    response = entity_extraction_chain.invoke(input)
    entites = json.loads(response.strip())
    print(entites)
    triplets = get_triplets_by_entites(entites)
    print(json.dumps(triplets, indent=4, ensure_ascii=False))
    context = '\n'.join(f"{i+1}. {str(triplet)}" for i, triplet in enumerate(triplets))
    # triplets = '\n'.join(str(triplet) for triplet in triplets)
    input = {'question': query, 'context': context}
    response = answer_chain.invoke(input)
    return response.strip()

In [7]:
query = """FPT AI là gì?"""
answer = get_answer(query)
print(answer)

['FPT AI']
[
    "FPT AI có giải pháp FPT AI Chat, nền tảng hội thoại thông minh cho phép tạo lập và quản lý chatbot tự động giao tiếp hai chiều với khách hàng.",
    "FPT AI cung cấp giải pháp FPT AI Engage, nền tảng trợ lý ảo tổng đài thực hiện cuộc gọi tự động hai chiều với giọng nói tự nhiên.",
    "FPT AI sở hữu giải pháp FPT AI Read, nền tảng nhận diện và trích xuất thông tin từ ảnh chụp với khả năng xử lý đa dạng mọi loại văn bản.",
    "FPT AI có giải pháp FPT AI Enhance, cho phép tổng đài tự động toàn diện quy trình quản lý chất lượng và theo dõi các chỉ số chuyên sâu theo thời gian thực.",
    "FPT AI cung cấp giải pháp FPT AI eKYC, khả năng định danh khách hàng hoàn chỉnh với nhiều cấp độ bảo mật.",
    "FPT AI giúp nâng cao hiệu suất xử lý công việc, mang lại sự kết hợp hoàn hảo giữa người và máy.",
    "FPT AI gắn kết khách hàng hiệu quả thông qua việc tiếp cận, tương tác và thu hút tối đa khách hàng.",
    "FPT AI kiến tạo giá trị mới cho khách hàng thông qua việc thiết l